In [1]:
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini")

In [2]:
from langchain_core.tools import tool
from datetime import datetime
import pytz


@tool
def get_current_time(timezone: str, location: str) -> str:
  """ 현재 시각을 반환하는 함수

    Args:
      timezone (str): 타임존(예: 'Asia/Seoul'). 실제 존재해야 한다.
      location (str): 지역명. 타임존은 모든 지명에 대응되지 않으므로 이후 llm 답변 생성에 사용된다.
  """
  tz = pytz.timezone(timezone)
  now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
  location_and_local_time = f'{timezone} ({location}) 현재 시각 {now}'
  print(location_and_local_time)
  return location_and_local_time

In [3]:
tools = [get_current_time, ]
tool_dict = {"get_current_time": get_current_time, }

llm_with_tools = llm.bind_tools(tools)

In [4]:
from langchain_core.messages import SystemMessage

messages = [
  SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
  HumanMessage("부산은 지금 몇 시야?"),
]

response = llm_with_tools.invoke(messages)
messages.append(response)

print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WM4dwQOufbs6qeiNh0n0yVDP', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 129, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--28f8d386-5209-4cad-b113-588d008530f6-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'loca

In [5]:
for tool_call in response.tool_calls:
  selected_tool = tool_dict[tool_call["name"]]
  print(tool_call["args"])
  tool_msg = selected_tool.invoke(tool_call)
  messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': 'Busan'}
Asia/Seoul (Busan) 현재 시각 2025-10-14 20:28:54


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WM4dwQOufbs6qeiNh0n0yVDP', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 129, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--28f8d386-5209-4cad-b113-588d008530f6-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'lo

In [6]:
llm_with_tools.invoke(messages)

AIMessage(content='부산의 현재 시각은 2025년 10월 14일 20시 28분 54초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 185, 'total_tokens': 214, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None}, id='run--ec6b0712-32d0-4683-8ff5-698af9eda822-0', usage_metadata={'input_tokens': 185, 'output_tokens': 29, 'total_tokens': 214, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from pydantic import BaseModel, Field


class StockHistoryInput(BaseModel):
  ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
  period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")

In [8]:
import yfinance as yf


@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
  """ 주식 종목의 가격 데이터를 조회하는 함수"""
  stock = yf.Ticker(stock_history_input.ticker)
  history = stock.history(period=stock_history_input.period)
  history_md = history.to_markdown()
  return history_md


tools = [get_current_time, get_yf_stock_history, ]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history, }

llm_with_tools = llm.bind_tools(tools)

In [9]:
messages.append(HumanMessage("테슬라는 한 달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_lNctHDM8b8VtONpzgtrakTbP', 'function': {'arguments': '{"stock_history_input":{"ticker":"TSLA","period":"1mo"}}', 'name': 'get_yf_stock_history'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 276, 'total_tokens': 303, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--d8d2f601-2e8e-489e-9fe5-4dac9c850180-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_lNctHDM8b8VtONpzgtrakTbP', 'type': 'tool_call'}] usage_metadata={'input_tokens': 276, 'output_tokens': 27, 'total_tokens': 303, 'input_token_det

In [10]:
for tool_call in response.tool_calls:
  selected_tool = tool_dict[tool_call["name"]]
  print(tool_call["args"])
  tool_msg = selected_tool.invoke(tool_call)
  messages.append(tool_msg)
  print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-09-15 00:00:00-04:00 | 423.13 | 425.7  | 402.43 |  410.04 | 1.63824e+08 |           0 |              0 |\n| 2025-09-16 00:00:00-04:00 | 414.5  | 423.25 | 411.43 |  421.62 | 1.04286e+08 |           0 |              0 |\n| 2025-09-17 00:00:00-04:00 | 415.75 | 428.31 | 409.67 |  425.86 | 1.06134e+08 |           0 |              0 |\n| 2025-09-18 00:00:00-04:00 | 428.87 | 432.22 | 416.56 |  416.85 | 9.04545e+07 |           0 |              0 |\n| 2025-09-19 00:00:00-04:00 | 421.82 | 429.47 | 421.72 |  426.07 | 9.3131e+07  |           0 |              0 |\n| 2025-09-22 00:00:00-04:00 | 431.11 | 444.98 | 429.13 |  434.21 | 9.71088e+07 |           0 |              0 |\n| 2025-09-23 00:00:00-04:0

In [11]:
llm_with_tools.invoke(messages)

AIMessage(content='한 달 전인 2025년 9월 15일 테슬라(TSLA)의 주가는 410.04달러로 마감했습니다. 현재 주가는 435.90달러입니다. \n\n따라서, 테슬라의 주가는 한 달 전보다 올랐습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 1574, 'total_tokens': 1640, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None}, id='run--e5ef9834-27d4-44d5-aa28-78e3412b01c0-0', usage_metadata={'input_tokens': 1574, 'output_tokens': 66, 'total_tokens': 1640, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
for c in llm.stream([HumanMessage("현재 Kpop 차트 1위곡")]):
  print(c.content, end='|')

|현재| K|-pop| 차|트| |1|위| 곡|에| 대한| 정보|는| 실|시간|으로| 변|동|되|므로|,| 정확|한| 정보를| 제공|하기| 위해|서는| 최신| 음악| 차|트를| 확인|해야| 합니다|.| 주요| K|-pop| 차|트|인| 가|온| 차|트|나| Billboard| K|-pop| 차|트를| 참고|하|시면| 최근| |1|위| 곡|을| 확인|할| 수| 있습니다|.| 추가|적으로| 궁|금|한| 내용|이나| 다른| 질문|이| 있으|시면| 말씀|해| 주세요|!||

In [14]:
messages = [
  SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
  HumanMessage("부산은 지금 몇 시야?"),
]

response = llm_with_tools.stream(messages)

is_first = True
for chunk in response:
  print("chunk type: ", type(chunk))
  if is_first:
    is_first = False
    gathered = chunk
  else:
    gathered += chunk

  print("content : ", gathered.content, "tool_call_chunk", gathered.tool_calls)

messages.append(gathered)

chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content :   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_eHCcRzMqNIZ24ywQl6YX22XK', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content :   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_eHCcRzMqNIZ24ywQl6YX22XK', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content :   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_eHCcRzMqNIZ24ywQl6YX22XK', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content :   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_eHCcRzMqNIZ24ywQl6YX22XK', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content :   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_eHCcRzMqNIZ24ywQl6YX22XK', 'type': 't

In [15]:
gathered

AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_eHCcRzMqNIZ24ywQl6YX22XK', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559'}, id='run--140eac43-4bc1-4aad-b259-0e3c6b6b17e9', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_eHCcRzMqNIZ24ywQl6YX22XK', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_eHCcRzMqNIZ24ywQl6YX22XK', 'index': 0, 'type': 'tool_call_chunk'}])

In [16]:
for tool_call in gathered.tool_calls:
  selected_tool = tool_dict[tool_call["name"]]
  print(tool_call["args"])
  tool_msg = selected_tool.invoke(tool_call)
  messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재 시각 2025-10-14 20:47:41


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_eHCcRzMqNIZ24ywQl6YX22XK', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559'}, id='run--140eac43-4bc1-4aad-b259-0e3c6b6b17e9', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_eHCcRzMqNIZ24ywQl6YX22XK', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_eHCcRzMqNIZ24ywQl6YX22XK', 'index': 0, 'type': 'tool_call_chunk'}]),
 ToolMessage(content='Asia/Seoul (부산) 현재 시각 2025-10-14

In [17]:
for c in llm_with_tools.stream(messages):
  print(c.content, end='|')

|부|산|은| 지금| |202|5|년| |10|월| |14|일| |20|시| |47|분|입니다|.||